<a href="https://colab.research.google.com/github/myy04/ResNet-CIFAR10/blob/main/hw2_1_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple CNN with PyTorch

In this notebook example, we will walk through how to train a simple CNN to classify images.

We will rely on the following modules, including torch and torchvision.

In [22]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

## 1. Data Loader

The first step is to create a data loader.

A data loader can be treated as a list (or iterator, technically). Each time it will provide a minibatch of (img, label) pairs.

Please wait till the number "2" apppears in the left "In[ ]" for the data to be fully downloaded, or execute this part again to see "Files already downloaded and verified".

In [23]:
# Choose a dataset -- MNIST for example
dataset = datasets.MNIST(root='data', train=True, download=True)

# Set how the input images will be transformed
dataset.transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.1307,], std=[0.3081,])
])

# Create a data loader
train_loader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=1)

# Show the shape of a batch.

print("The shape of one batch is {}".format((next(iter(train_loader)))[0].size()))

The shape of one batch is torch.Size([256, 1, 28, 28])


## 2. Model

The second step is to define our model. This part is left to be filled by yourself.

The way to define a model can be found at [this tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py).

The __requirements__ are as the following:

* Define the first convolutional layer with channel size = 5, kernel size = 3 and stride = 2, padding = 1.
* Define the second convolutional layer with channel size = 8, kernel size = 3 and stride = 1, padding = 1.
* Use max pooling layer with stride = 2 between the two convolution layers.
* Define the FC layer(s) and finally return a tensor with shape torch.Size([256, 10]). (Use torch.view to reshape tensors. You can try any number of FC layers).
* Use ReLU activation between any two layers.

In [24]:
"""
Define the Model. The requirements are:
    Define the first convolutional layer with channel size = 5, kernel size = 3 and stride = 2, padding = 1.
    Define the second convolutional layer with channel size = 8, kernel size = 3 and stride = 1, padding = 1.
    Use max pooling layer with stride = 2 between the two convolution layers.
    Define the FC layer(s) and finally return a tensor with shape torch.Size([256, 10]). (Use torch.view to reshape tensors. You can try any number of FC layers).
    Use ReLU activation between any two layers.
"""

import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, stride=2, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=5, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.fc_layers = nn.ModuleList([
            nn.Linear(8 * 7 * 7, 256),
            nn.Linear(256, 128),
            nn.Linear(128, 10)
        ])

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = x.view(-1, 8 * 7 * 7)
        for fc in self.fc_layers[:-1]:
            x = F.relu(fc(x))
        x = self.fc_layers[-1](x)
        return x

device = torch.device('cuda')
model = SimpleCNN().to(device)


#
#
#

## 3. Loss and Optimizer

The third step is to define the loss function and the optimization algorithm.

* Define the __criterion__ to be Cross Entropy Loss.
* Define the __optimizer__ to be SGD with momentum factor 0.9 and weight_decay 5e-4.

Information can be found at PyTorch documents.

In [25]:
#############################################################################
# TODO: Define the criterion to be Cross Entropy Loss.                      #
#       Define the optimizer to be SGD with momentum factor 0.9             #
#       and weight_decay 5e-4.
# You may change the learning rate.                                      #
#############################################################################


import torch.optim as optim

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=5e-4)


#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################

## 4. Start training

The next step is to start the training process.

In [26]:
def train(epoch):
    model.train()  # Set the model to be in training mode
    for batch_index, (inputs, targets) in enumerate(train_loader):
        # Forward
        # You may change 'device' of inputs here !!!
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        if batch_index % 10 == 0:
            print('epoch {}  batch {}/{}  loss {:.3f}'.format(
                epoch, batch_index, len(train_loader), loss.item()))

        # Backward
        optimizer.zero_grad()  # Set parameter gradients to zero
        loss.backward()        # Compute (or accumulate, actually) parameter gradients
        optimizer.step()       # Update the parameters


In [27]:
# Choose a dataset -- MNIST for example
dataset = datasets.MNIST(root='data', train=False, download=True)

# Set how the input images will be transformed
dataset.transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.1307, ], std=[0.3081, ])
])

# Create a data loader
test_dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=1)

def test(dataloader):
    model.eval()

    # Evaluate your model on the test dataset
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to('cuda'), labels.to('cuda')
            outputs = model(images)
            _, predicted = torch.max(outputs.data, -1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Print the accuracy and loss
    accuracy = correct / total
    print('Accuracy:', accuracy)

In [28]:
for epoch in range(0, 6):
    train(epoch)
    # You may validate model here

    test(test_dataloader)  # Call the test function to evaluate the model

epoch 0  batch 0/235  loss 2.306
epoch 0  batch 10/235  loss 2.302
epoch 0  batch 20/235  loss 2.301
epoch 0  batch 30/235  loss 2.297
epoch 0  batch 40/235  loss 2.294
epoch 0  batch 50/235  loss 2.290
epoch 0  batch 60/235  loss 2.282
epoch 0  batch 70/235  loss 2.275
epoch 0  batch 80/235  loss 2.261
epoch 0  batch 90/235  loss 2.229
epoch 0  batch 100/235  loss 2.184
epoch 0  batch 110/235  loss 2.055
epoch 0  batch 120/235  loss 1.706
epoch 0  batch 130/235  loss 1.071
epoch 0  batch 140/235  loss 0.777
epoch 0  batch 150/235  loss 0.645
epoch 0  batch 160/235  loss 0.593
epoch 0  batch 170/235  loss 0.537
epoch 0  batch 180/235  loss 0.545
epoch 0  batch 190/235  loss 0.490
epoch 0  batch 200/235  loss 0.468
epoch 0  batch 210/235  loss 0.417
epoch 0  batch 220/235  loss 0.575
epoch 0  batch 230/235  loss 0.423
Accuracy: 0.8827
epoch 1  batch 0/235  loss 0.321
epoch 1  batch 10/235  loss 0.270
epoch 1  batch 20/235  loss 0.428
epoch 1  batch 30/235  loss 0.309
epoch 1  batch 40/2

## 5. What's next?

We have sketched a simple framework for training CNNs. There are a few more functions yet to be completed.

  - Use gpu and cudnn
  - Do validation after each epoch
  - Adjust the learning rate

Please read the ppt carefully in class.